In [ ]:
!pip install pytube yt_dlp youtube-transcript-api

In [ ]:
from pytube import Playlist
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

def get_video_ids(playlist_url):
    """재생목록에서 영상 ID 목록을 가져오는 함수"""
    try:
        playlist = Playlist(playlist_url)
        # video_urls에서 ID만 추출
        video_ids = [url.split('=')[1] for url in playlist.video_urls]
        return video_ids
    except Exception as e:
        print(f"재생목록 가져오기 오류: {e}")
        return []

def get_transcript_info(video_ids):
    """영상 ID 목록으로부터 자막 정보를 가져오는 함수"""
    transcript_data = []

    for video_id in video_ids:
        try:
            # 자막 가져오기 시도
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

            # 전체 텍스트 결합
            full_text = ' '.join([entry['text'] for entry in transcript])

            # 첫 번째와 마지막 자막의 시간으로 영상 길이 계산
            video_duration = transcript[-1]['start'] + transcript[-1]['duration']

            video_info = {
                'video_id': video_id,
                'url': f'https://youtube.com/watch?v={video_id}',
                'duration_seconds': video_duration,
                'duration_formatted': f"{int(video_duration//60)}:{int(video_duration%60):02d}",
                'transcript_text': full_text,
                'transcript_available': True
            }

        except Exception as e:
            # 자막을 가져올 수 없는 경우
            video_info = {
                'video_id': video_id,
                'url': f'https://youtube.com/watch?v={video_id}',
                'duration_seconds': None,
                'duration_formatted': None,
                'transcript_text': None,
                'transcript_available': False
            }
            print(f"영상 {video_id} 자막 가져오기 실패: {str(e)}")

        transcript_data.append(video_info)
        print(f"처리완료: {video_id}")

    return pd.DataFrame(transcript_data)

# 메인 실행 코드
playlist_url = "https://www.youtube.com/watch?v=g_qxoznfa7E&list=PLOXw6I10VTv9lin5AzsHAHCTrC7BdVdEM"

# 1. 재생목록에서 영상 ID 가져오기
video_ids = get_video_ids(playlist_url)
print(f"총 {len(video_ids)}개의 영상 ID를 가져왔습니다.")

# 2. 각 영상의 자막 정보 가져오기
if video_ids:
    df = get_transcript_info(video_ids)

    # 결과 출력
    print("\n데이터프레임 미리보기:")
    print(df)

    # CSV 파일로 저장
    df.to_csv('youtube_transcripts.csv', index=False, encoding='utf-8-sig')
    print("\n데이터가 'youtube_transcripts.csv' 파일로 저장되었습니다.")

In [ ]:
df

In [ ]:
from openai import OpenAI
import time
import pandas as pd

def translate_text(client, text):
    """OpenAI API를 사용하여 텍스트를 번역하는 함수"""
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a professional translator. Translate the following English text to Korean. Maintain the original meaning and nuance as much as possible."},
                {"role": "user", "content": text}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"번역 중 오류 발생: {e}")
        return None

def translate_dataframe(df):
    """DataFrame의 transcript_text 컬럼을 번역하는 함수"""
    # OpenAI 클라이언트 설정
    client = OpenAI()

    # 새로운 컬럼 추가
    df['transcript_text_ko'] = None

    # 각 행에 대해 번역 수행
    for idx, row in df.iterrows():
        if pd.isna(row['transcript_text']):
            continue

        print(f"번역 중... ({idx + 1}/{len(df)})")

        # 텍스트가 너무 길 경우 청크로 나누기 (OpenAI API 토큰 제한 고려)
        text_chunks = [row['transcript_text'][i:i+4000] for i in range(0, len(row['transcript_text']), 4000)]
        translated_chunks = []

        for chunk in text_chunks:
            # 번역 수행
            translated_text = translate_text(client, chunk)

            print(translated_text[:300])

            if translated_text:
                translated_chunks.append(translated_text)

            # API 호출 간 딜레이 (rate limit 방지)
            time.sleep(1)

        # 번역된 청크들을 하나로 합치기
        full_translation = ' '.join(translated_chunks)

        # DataFrame 업데이트
        df.at[idx, 'transcript_text_ko'] = full_translation

        # 중간 저장 (오류 발생 시 복구를 위해)
        if (idx + 1) % 5 == 0:
            df.to_csv('youtube_transcripts_with_translation.csv', index=False, encoding='utf-8-sig')
            print(f"중간 저장 완료 - {idx + 1}개 처리됨")

    return df

# 기존 DataFrame 불러오기 (이전 코드에서 저장한 CSV 파일)
df = pd.read_csv('youtube_transcripts.csv')

# 번역 실행
translated_df = translate_dataframe(df)

# 최종 결과 저장
translated_df.to_csv('youtube_transcripts_with_translation.csv', index=False, encoding='utf-8-sig')
print("번역 완료! 결과가 'youtube_transcripts_with_translation.csv' 파일로 저장되었습니다.")

# 결과 미리보기
print("\n번역 결과 미리보기:")
print(translated_df[['video_id', 'transcript_text', 'transcript_text_ko']].head())